In [5]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import LdaModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


In [6]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Muzhen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
#df1= pd.read_csv('reddit review.csv', encoding="cp1252")
#df1.head()
df1= pd.read_csv('/Users/Muzhen/Downloads/reddit review.csv', encoding="cp1252")
df1.head()

,agree #,review_text
0,132,"""Kids grow really fast and a lot of people bu..."
1,3,"""A lot of the op shops in Perth tend to check..."
2,21,"""Really depends on the area and the staff's g..."
3,2,"""They were half of half and also on clearance..."
4,31,"""How our used clothing stores mark everything..."


In [8]:
df_1=df1
print(df1.size)

892


In [9]:
def word_count(sentence):
    translate = sentence.maketrans({char: None for char in "'.,:*!"})
    cleaned_words = sentence.lower().translate(translate).split()
    word_counter = {}
    for word in cleaned_words:
        if word in word_counter:
            word_counter[word] += 1
        else:
            word_counter[word] = 1
    return word_counter
example = word_count("It's a man, it's a plane, it's superman!")
print (example)

{'its': 3, 'a': 2, 'man': 1, 'plane': 1, 'superman': 1}


In [10]:
example = word_count(df_1.loc[0,"review_text"])
print (df_1.loc[0,"review_text"])
print (example)
print ("")

example = word_count(df_1.loc[1,"review_text"])
print (df_1.loc[1,"review_text"])
print (example)
print ("")

example = word_count(df_1.loc[2,"review_text"])
print (df_1.loc[2,"review_text"])
print (example)
print ("")


example = word_count(df_1.loc[4,"review_text"])
print (df_1.loc[4,"review_text"])
print (example)
print ("")

 "Kids grow really fast and a lot of people buy a few season's worth", 
{'"kids': 1, 'grow': 1, 'really': 1, 'fast': 1, 'and': 1, 'a': 2, 'lot': 1, 'of': 1, 'people': 1, 'buy': 1, 'few': 1, 'seasons': 1, 'worth"': 1}

 "A lot of the op shops in Perth tend to check the original price of any brand name items online now and price accordingly, mostly to cash in on the profits that others are making by buying their goods cheap and flogging them on \"Instagram boutiques\"", 
{'"a': 1, 'lot': 1, 'of': 2, 'the': 3, 'op': 1, 'shops': 1, 'in': 2, 'perth': 1, 'tend': 1, 'to': 2, 'check': 1, 'original': 1, 'price': 2, 'any': 1, 'brand': 1, 'name': 1, 'items': 1, 'online': 1, 'now': 1, 'and': 2, 'accordingly': 1, 'mostly': 1, 'cash': 1, 'on': 2, 'profits': 1, 'that': 1, 'others': 1, 'are': 1, 'making': 1, 'by': 1, 'buying': 1, 'their': 1, 'goods': 1, 'cheap': 1, 'flogging': 1, 'them': 1, '\\"instagram': 1, 'boutiques\\""': 1}

 "Really depends on the area and the staff's general level of knowledge.

In [11]:
data = df_1.review_text.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:4])

[' "Kids grow really fast and a lot of people buy a few seasons worth", ',
 ' "A lot of the op shops in Perth tend to check the original price of any '
 'brand name items online now and price accordingly, mostly to cash in on the '
 'profits that others are making by buying their goods cheap and flogging them '
 'on \\"Instagram boutiques\\"", ',
 ' "Really depends on the area and the staffs general level of knowledge. A '
 'lot of second hand stores in Australia are run by charities/volunteers, so '
 'mileage varies when it comes to pricing.", ',
 ' "They were half of half and also on clearance. It was my lucky day lol", ']


In [12]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:2])

[['kids', 'grow', 'really', 'fast', 'and', 'lot', 'of', 'people', 'buy', 'few', 'seasons', 'worth'], ['lot', 'of', 'the', 'op', 'shops', 'in', 'perth', 'tend', 'to', 'check', 'the', 'original', 'price', 'of', 'any', 'brand', 'name', 'items', 'online', 'now', 'and', 'price', 'accordingly', 'mostly', 'to', 'cash', 'in', 'on', 'the', 'profits', 'that', 'others', 'are', 'making', 'by', 'buying', 'their', 'goods', 'cheap', 'and', 'flogging', 'them', 'on', 'instagram', 'boutiques']]


In [13]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(bigram_mod[data_words[2]])

['really', 'depends', 'on', 'the', 'area', 'and', 'the', 'staffs', 'general', 'level', 'of', 'knowledge', 'lot', 'of', 'second_hand', 'stores', 'in', 'australia', 'are', 'run', 'by', 'charities', 'volunteers', 'so', 'mileage', 'varies', 'when', 'it', 'comes', 'to', 'pricing']


In [14]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [15]:
import nltk
from nltk.corpus import stopwords

In [16]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm')

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])
print(data_lemmatized[1:2])

[['kid', 'grow', 'really', 'fast', 'lot', 'people', 'buy', 'season', 'worth']]
[['lot', 'op', 'shop', 'check', 'original', 'price', 'brand', 'name', 'item', 'online', 'price', 'accordingly', 'mostly', 'cash', 'profit', 'other', 'make', 'buy', 'good', 'cheap', 'flog', 'instagram', 'boutique']]


In [17]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:4])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)], [(0, 1), (4, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1)], [(4, 1), (6, 1), (26, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1)], [(41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1)]]


In [24]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.02,
                                           eta=0.05,
                                           per_word_topics=True)

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.3452076126185906


In [25]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=7, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                            eta=0.05,
                                           per_word_topics=True)



In [28]:
pprint(lda_model.print_topics(num_words=8))

[(0,
  '0.015*"plastic" + 0.014*"brand" + 0.012*"years" + 0.011*"recycled" + 0.011*"eco-friendly" + '
  '0.011*"marketing" + 0.010*"resale" + 0.009*"use"'),
 (1,
  '0.022*"second" + 0.018*"store" + 0.014*"good" + 0.010*"hand" + 0.009*"thrift" + '
  '0.008*"circular" + 0.008*"reuse" + 0.008*"search"'),
 (2,
  '0.018*"fast" + 0.012*"buy" + 0.011*"fashion" + 0.010*"get" + '0.009*"claims" + '
  '0.009*"brand" + 0.008*"greenwashing" + 0.007*"company"'),
 (3,
  '0.019*"buy" + 0.014*"used" + 0.013*"new" + 0.011*"quality" + 0.011*"decision" + '
  '0.010*"purchase" + 0.010*"value" + 0.010*"durability"'),
 (4,
  '0.015*"leather" + 0.013*"vegan" + 0.012*"alternative" + 0.011*"reduce" + '
  '0.010*"claim" + 0.009*"luxury" + 0.009*"options" + 0.008*"product"'),
 (5,
  '0.013*"carbon" + 0.011*"ethical" + 0.010*"need" + 0.009*"offset" + 0.009*"environment" + '
  '0.009*"emiison" + 0.009*"promise" +  0.0009*"supply"'),
 (6,
  '0.013*"cotton" + 0.011*"like" + 0.010*"thrift" + 0.009*"fabric" + 0.009*"na